In [1]:
import os
import numpy as np 
import pandas as pd 

In [2]:
!wget -P ./../../../dataset/ -nc https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/airquality_nmhc.csv

File './../../../dataset/airquality_nmhc.csv' already there; not retrieving.



In [3]:
path = "./../../../dataset/airquality_nmhc.csv"
dataset_name = os.path.splitext(os.path.basename(path))[0]
data = pd.read_csv(path, index_col=0)
print("shape of data: ", data.shape)
data.head() 

shape of data:  (887, 12)


,true_error,time_slot,day_of_week,T,RH,AH,PT08.S1(CO),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),true_label,given_label
0,0,evening,wednesday,13.6,48.9,0.7578,1360.0,1056.0,1692.0,1268.0,0.6855,1.1138
1,0,evening,wednesday,13.3,47.7,0.7255,1292.0,1174.0,1559.0,972.0,0.5119,1.0169
2,0,evening,wednesday,11.9,54.0,0.7502,1402.0,1140.0,1555.0,1074.0,0.4022,0.9998
3,0,evening,wednesday,11.0,60.0,0.7867,1376.0,1092.0,1584.0,1203.0,0.3656,1.0094
4,0,night,wednesday,11.2,59.6,0.7888,1272.0,1205.0,1490.0,1110.0,0.2331,0.8902


In [4]:
training_data = data.drop(['true_label', 'true_error'], axis=1)
training_data.head(2)

,time_slot,day_of_week,T,RH,AH,PT08.S1(CO),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),given_label
0,evening,wednesday,13.6,48.9,0.7578,1360.0,1056.0,1692.0,1268.0,1.1138
1,evening,wednesday,13.3,47.7,0.7255,1292.0,1174.0,1559.0,972.0,1.0169


In [5]:
y = training_data['given_label']
X = training_data.drop(['given_label'], axis=1)

###### Add your training code here ###############
# model = 
# model_fit = model.fit(X,y)
# predictions = model.predict(X)

##### saving the predictions ####################
# savepath = "./../predictions/model_predictions.npy"
# np.save(savepath, np.array(predictions))

## Section section is relevant only if using AutoGluon

Uncomment the code in following cells and run to generate output and save to predictions folder. 

### set up arguments 

In [6]:
from autogluon.tabular import TabularPredictor

data_without_label = training_data.drop(['given_label'], axis=1)
savepath = "./../trained_models/"
if not os.path.isdir(savepath):
    os.mkdir(savepath)

predictorArgs = {
                    "path"             : savepath, 
                    "label"            : 'given_label',
                    "eval_metric"      : 'r2',
                    "problem_type"     : 'regression'
                }

hyperparameters = { 
                    'GBM'   : {}, 
                    'FASTAI': {}, 
                    'RF'    : {'criterion': 'squared_error', 
                                'ag_args': {'name_suffix': 'MSE', 
                                'problem_types': ['regression', 'quantile']}}
                    }

### Normal Model fitting

In [7]:
predictor = TabularPredictor(**predictorArgs)
predictor.fit(training_data, hyperparameters=hyperparameters)
leaderboard = predictor.leaderboard(training_data, silent=True)
leaderboard

Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    887
Train Data Columns: 9
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2007.24 MB
	Train Data (Original)  Memory Usage: 0.16 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtype

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE,0.995176,0.981793,0.044586,0.017882,0.266170,0.044586,0.017882,0.266170,1,True,2
1,LightGBM,0.992729,0.983828,0.005206,0.002778,2.507491,0.005206,0.002778,2.507491,1,True,1
2,WeightedEnsemble_L2,0.992654,0.990100,0.022059,0.009886,4.505725,0.002968,0.000195,0.027995,2,True,4
3,NeuralNetFastAI,0.991153,0.989621,0.013885,0.006913,1.970239,0.013885,0.006913,1.970239,1,True,3


### Cross Validation fitting

In [8]:
predictor_cv = TabularPredictor(**predictorArgs) 
predictor_cv.fit(training_data, presets="best_quality", num_stack_levels= 0, hyperparameters=hyperparameters)
leaderboard_cv = predictor_cv.leaderboard(training_data, silent=True)
leaderboard_cv

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    887
Train Data Columns: 9
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1725.32 MB
	Train Data (Original)  Memory Usage: 0.16 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerato

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE_BAG_L1,0.998138,0.986191,0.063023,0.055056,0.306324,0.063023,0.055056,0.306324,1,True,2
1,WeightedEnsemble_L2,0.995368,0.992934,0.192724,0.220934,13.257514,0.001114,0.000201,0.067896,2,True,4
2,NeuralNetFastAI_BAG_L1,0.994407,0.992756,0.128587,0.165677,12.883295,0.128587,0.165677,12.883295,1,True,3
3,LightGBM_BAG_L1,0.993259,0.977686,0.092906,0.034590,18.157121,0.092906,0.034590,18.157121,1,True,1


### predicting and saving arrays 

In [ ]:
models = {  'normal'            : list(leaderboard['model']), 
            'crossValidation'   : list(np.append(leaderboard_cv['model'], ['oof']))}

predictors = { 'normal'          : predictor, 
               'crossValidation' : predictor_cv}

# check if path is available, else create it. 
pred_path = "./../predictions/"
if not os.path.isdir(pred_path):
    os.mkdir(pred_path)

for model_type in models.keys():
    if model_type == "normal": 
        for model in models[model_type]:
            savepath = os.path.join(pred_path, model+".npy")
            predictions = predictors[model_type].predict(data_without_label, model=model)
            np.save(savepath, np.array(predictions))
    elif model_type == "crossValidation":
        for model in models[model_type]:
            if model == "oof":
                predictions = predictors[model_type].get_oof_pred()
                savepath = os.path.join(pred_path, model+".npy")
            elif model in models['normal']:
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+"_CV.npy")
            else: 
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+".npy")

            np.save(savepath, np.array(predictions))